# Introduction

This tutorial provides instructions on nhow to use `city_metrix` to calcaulte indicators based on user-specific geometry.

Every 'indicator' is defined as separate python function:

| Indicator name | function name | Parameters | Method |
| ---- | ---- | ---- | ---- |
| Average tree cover | `mean_tree_cover()`  | | |
| Percent of built land without tree cover | `built_land_without_tree_cover()` | | |

# Setting

In [1]:
import os
import geopandas as gpd
from rasterio.plot import show
import rasterio
import pandas as pd

In [2]:
# update the wd path to be able to laod the module
os.chdir('../..') 
os.getcwd()

'/Users/jt/dev/cities-cif'

In [3]:
os.environ['GCS_BUCKET']='gee-exports'
os.environ['GOOGLE_APPLICATION_USER']='developers@citiesindicators.iam.gserviceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='C:\\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities-indicators-framework\citymetrix\credentials-citiesindicators.json'

# Get boundaries

In [3]:
# load boundary of a region of interest
boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4union.geojson'
city_gdf_aoi = gpd.read_file(boundary_path, driver='GeoJSON')
city_gdf_aoi.head()

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."


In [4]:
# load boundary of sub-georgaphy
boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4.geojson'
city_gdf_sub = gpd.read_file(boundary_path, driver='GeoJSON')
city_gdf_sub.head()

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.40125 -12.96457, -38.40126..."
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.39898 -12.96269, -38.39898..."
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.37346 -12.93345, -38.37386..."
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.41741 -12.97578, -38.41746..."
4,BRA-Salvador_ADM4_5,ADM4,Jardim Armação,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.43383 -12.98742, -38.43386..."


# LND-2 Average Tree Cover

In [5]:
from city_metrix.metrics import mean_tree_cover

Authenticating to GEE with configured credentials file.


In [6]:
from city_metrix import mean_tree_cover

## City level - one city

In [9]:
%%time

# for one geography
city_gdf_aoi["mean_tree_cover"] = mean_tree_cover(city_gdf_aoi) 
city_gdf_aoi

Extracting layer tree cover from Google Earth Engine:
CPU times: user 6.65 s, sys: 1.12 s, total: 7.77 s
Wall time: 12.5 s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.283668


## Sub-city level

In [8]:
%%time

# for sub geographies
city_gdf_sub["mean_tree_cover"] = mean_tree_cover(city_gdf_sub) 
city_gdf_sub.head()

Extracting tree cover layer in bbox [-38.6469891  -13.0171475  -38.30455157 -12.75628844]:
[########################################] | 100% Completed | 14.55 s
CPU times: total: 17.1 s
Wall time: 17.8 s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.40125 -12.96457, -38.40126...",0.523927
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.39898 -12.96269, -38.39898...",0.433088
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.37346 -12.93345, -38.37386...",0.274104
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.41741 -12.97578, -38.41746...",0.157694
4,BRA-Salvador_ADM4_5,ADM4,Jardim Armação,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.43383 -12.98742, -38.43386...",0.210874


## Multiple cities

In [9]:
# Calculate indicator for multiple regions of interest

boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4union.geojson'
city_Salvador = gpd.read_file(boundary_path, driver='GeoJSON')

boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-IDN-Jakarta-ADM4union.geojson'
city_Jakarta = gpd.read_file(boundary_path, driver='GeoJSON')

cities = pd.concat([city_Salvador, city_Jakarta])
cities = cities.reset_index(drop=True)
cities

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."
1,IDN-Jakarta_ADM-4-union_1,ADM-4-union,IDN-Jakarta,IDN-Jakarta,2022-06-27,"MULTIPOLYGON (((106.78141 -6.31616, 106.78124 ..."


In [10]:
cities["mean_tree_cover"] = mean_tree_cover(cities)

Extracting tree cover layer in bbox [-38.6469891  -13.0171475  -38.30455157 -12.75628844]:
[########################################] | 100% Completed | 16.77 s
Extracting tree cover layer in bbox [106.685589   -6.3650478 106.8530109  -6.0891749]:
[########################################] | 100% Completed | 9.38 ss
Extracting tree cover layer in bbox [106.8530109  -6.3744575 106.973975   -6.0895823]:
[########################################] | 100% Completed | 8.75 ss


In [11]:
cities

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",0.283667
1,IDN-Jakarta_ADM-4-union_1,ADM-4-union,IDN-Jakarta,IDN-Jakarta,2022-06-27,"MULTIPOLYGON (((106.78141 -6.31616, 106.78124 ...",0.159044


# HEA-4 Percent of built land without tree cover

In [14]:
from city_metrix.metrics import built_land_without_tree_cover

In [15]:
%%time

# for one geography
city_gdf_aoi["built_land_without_tree_cover"] = built_land_without_tree_cover(city_gdf_aoi) 
city_gdf_aoi

Extracting ESA world cover layer in bbox [-38.6469891  -13.0171475  -38.30455157 -12.75628844]:
[########################################] | 100% Completed | 9.68 ss


UnboundLocalError: local variable 'stats_funcs_dict' referenced before assignment

# HEA-3 Built land with Low Surface reflectivity

In [12]:
from city_metrix.metrics import built_land_with_low_surface_reflectivity

In [13]:
%%time

city_gdf_aoi["built_land_with_low_surface_reflectivity"] = built_land_with_low_surface_reflectivity(city_gdf_aoi) 
city_gdf_aoi

Extracting ESA world cover layer:
[########################################] | 100% Completed | 8.21 ss
Calculating albedo layer:
[########################################] | 100% Completed | 84.36 s
Extracting ESA world cover layer:
[########################################] | 100% Completed | 8.32 ss
CPU times: total: 30.5 s
Wall time: 1min 49s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover,built_land_without_tree_cover,built_land_with_low_surface_reflectivity
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366734,0.907382,0.786552


# HEA-2 Built Land With High LST

In [14]:
from city_metrix.metrics import built_land_with_high_land_surface_temperature

In [15]:
%%time

city_gdf_aoi["built_land_with_high_land_surface_temperature"] = built_land_with_high_land_surface_temperature(city_gdf_aoi) 
city_gdf_aoi

Extracting ESA world cover layer:
[########################################] | 100% Completed | 8.65 ss
Calculating land surface temperature layer:
[########################################] | 100% Completed | 2.73 ss
Extracting ESA world cover layer:
[########################################] | 100% Completed | 9.16 ss
CPU times: total: 14.8 s
Wall time: 37.7 s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover,built_land_without_tree_cover,built_land_with_low_surface_reflectivity,built_land_with_high_land_surface_temperature
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366734,0.907382,0.786552,0.0961


# Percent of Natural areas

In [18]:
from city_metrix.metrics import natural_areas

In [ ]:
%%time

# for one geography
city_gdf_aoi["natural_areas"] = natural_areas(city_gdf_aoi) 
city_gdf_aoi

# Open Space in built up land

In [21]:
from city_metrix.metrics import urban_open_space

In [22]:
%%time

# for one geography
city_gdf_aoi["urban_open_space"] = urban_open_space(city_gdf_aoi) 
city_gdf_aoi

Extracting ESA world cover layer:
[########################################] | 100% Completed | 12.48 s
Extracting ESA world cover layer:
[########################################] | 100% Completed | 9.08 ss
CPU times: total: 24 s
Wall time: 31.1 s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover,built_land_without_tree_cover,built_land_with_low_surface_reflectivity,built_land_with_high_land_surface_temperature,urban_open_space
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366734,0.907382,0.786552,0.0961,0.028378
